In [1]:
import os, sys
from pathlib import Path

def find_project_root(marker_dirs=('src','data')):
    p = Path().resolve()
    while True:
        if all((p / d).is_dir() for d in marker_dirs):
            return p
        if p.parent == p:
            raise RuntimeError(f"Could not find project root containing {marker_dirs}")
        p = p.parent

project_root = find_project_root()
os.chdir(project_root)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("✔️ Project root:", project_root)


✔️ Project root: /Users/LeoWang/Desktop/CU/Risk/Risk_Project


In [2]:
# ── Cell 2: Load config, data, & calibrate ─────────────────────────
from risk_project.config         import STOCK_FILES, TARGET_NOTIONAL, TRADING_DAYS_YR
from risk_project.data_loader    import load_price_series
from risk_project.calibration    import estimate_mu_sigma, estimate_covariance_matrix

# Load prices
series = load_price_series(STOCK_FILES)

# Position sizing
positions = {
    sym: TARGET_NOTIONAL / series[sym].iloc[-1]
    for sym in series
}

# Calibrate mu & cov
mu  = {sym: estimate_mu_sigma(ps, trading_days_per_year=TRADING_DAYS_YR)[0]
       for sym, ps in series.items()}
cov = estimate_covariance_matrix(series, trading_days_per_year=TRADING_DAYS_YR)

print("Positions:", positions)
print("μ:", mu)
print("Cov matrix:\n", cov)


Positions: {'AAPL': 429.88565041698905, 'AMZN': 429.6270836913559}
μ: {'AAPL': 0.1753632543300528, 'AMZN': 0.2806626323807433}
Cov matrix:
           AAPL      AMZN
AAPL  0.196869  0.075444
AMZN  0.075444  0.303796


In [3]:
# ─── Cell 3: Compute & Print VaR/ES ────────────────────────────────
from risk_project.config      import P_VAR, HORIZON_DAYS, MC_PATHS, SEED, TRADING_DAYS_YR
from risk_project.var_es      import parametric_var_es, historical_var_es, monte_carlo_var_es

# 1) Parametric VaR & ES
pvar, pes = parametric_var_es(
    positions,
    series,
    mu,
    cov,
    p=P_VAR,
    horizon_days=HORIZON_DAYS,
    trading_days=TRADING_DAYS_YR,
)
print(f"Parametric 1-day VaR: ${pvar:,.2f}, ES: ${pes:,.2f}")

# 2) Historical VaR & ES
hvar, hes = historical_var_es(
    positions,
    series,
    p=P_VAR,
    horizon_days=HORIZON_DAYS,
    is_long=True,
)
print(f"Historical 1-day VaR: ${hvar:,.2f}, ES: ${hes:,.2f}")

# 3) Monte Carlo VaR & ES
mcvar, mces = monte_carlo_var_es(
    positions,
    series,
    mu,
    cov,
    p=P_VAR,
    horizon_days=HORIZON_DAYS,
    trading_days=TRADING_DAYS_YR,
    n_sims=MC_PATHS,
    seed=SEED,
)
print(f"MC 1-day VaR: ${mcvar:,.2f}, ES: ${mces:,.2f}")


Parametric 1-day VaR: $11,648.07, ES: $13,371.14
Historical 1-day VaR: $3,088.35, ES: $4,420.94
MC 1-day VaR: $11,310.55, ES: $13,107.62
